## Multiple Regression
* Simple Linear Regression:
* $$y = \beta_0 + \beta_1X$$
* Multiple Linear Regression:
* $$y = \beta_0 + \beta_1X_1 + \beta_2X_2 + ...$$
* Well studied field in statistics
* Focus will be on what is relevant for Data Science - practical and relevant for prediction

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
data = pd.read_csv('housing.data',delim_whitespace=True,header=None)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'housing.data'

In [ ]:
col = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']

In [ ]:
data.columns=col
data.head()

In [ ]:
x = data.iloc[0:,:-1]
x

In [ ]:
y = data['MEDV'].values

# Stats Model-1

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

### We need to add a constant term to allow statsmodel.api to calculate the bias / intercepts.

In [ ]:
constant = sm.add_constant(x)
constant.head()

In [ ]:
pd.DataFrame(constant)

In [ ]:
#sm.OLS?

In [ ]:
model = sm.OLS(y, constant)

In [ ]:
#lr=linearregression
lr = model.fit()

In [ ]:
lr.summary()

There are a lot of statistical tests and information. Mostly for the purpose of statistical analysis.

You do not need all of these for data science.

Data science focus is on prediction and having models that work on predicting real data. It is not concerned as much with  correct specifications of statistical problems.

## Model Statistical Outputs:

**Dep. Variable**: The dependent variable or target variable

**Model**: Highlight the model used to obtain this output. It is OLS here. Ordinary least squares / Linear regression

**Method**: The method used to fit the data to the model. Least squares

**No. Observations**: The number of observations

**DF Residuals**: The degrees of freedom of the residuals. Calculated by taking the number of observations less the number of parameters

**DF Model**: The number of estimated parameters in the model. In this case 13. The constant term is not included.


**R-squared**: This is the coefficient of determination. Measure of goodness of fit.
$$R^2=1-\frac{SS_{res}}{SS_{tot}}$$

> From [wiki](https://en.wikipedia.org/wiki/Coefficient_of_determination),

  > The total sum of squares, $SS_{tot}=\sum_i(y_i-\bar{y})^2$

  > The regression sum of squares (explained sum of squares), $SS_{reg}=\sum_i(f_i-\bar{y})^2$

  > The sum of squares of residuals (residual sum of squares), $SS_{res}=\sum_i(y_i-f_i)^2 = \sum_ie^2_i$

**Adj. R-squared**: This is the adjusted R-squared. It is the coefficient of determination adjusted by sample size and the number of parameters used.
$$\bar{R}^2=1-(1-R^2)\frac{n-1}{n-p-1}$$

> $p$ = The total number of explanatory variables not including the constant term

> $n$ = The sample size

**F-statistic**: A measure that tells you if you model is different from a simple average.

**Prob (F-statistic)**: This measures the significance of your F-statistic. Also called p-value of F-statistic. In statistics, p-value equal or lower than 0.05 is considered significant.

**AIC**: This is the Akaike Information Criterion. It evaluatess the model based on the model complexity and number of observations. The lower the better. 

**BIC**: This is the Bayesian Information Criterion. Similar to AIC, except it pushishes models with more parameters.

## Parameters Estimates and the Associated Statistical Tests

**coef**: The estimated coefficient. Note that this is just a point estimate.

**std err**: The standard error of the estimate of the coefficient. Another term for standard deviation

**t**: The t-statistic score. 

**P > |t|**: The p-value. A measure of the probability that the coefficient is different from zero.

**[95.0% Conf. Interval]**: The 95% confidence interval of the coefficient. Shown here as [0.025, 0.975], the lower and upper bound.

## Residual Tests

**Omnibus D'Angostino's test**: This is a combined statistical test for skewness and kurtosis.

**Prob(Omnibus)**: p-value of Omnibus test.

**Skewness**: This is a measure of the symmetry of the residuals around the mean. Zero if symmetrical. A positive value indicates a long tail to the right; a negative value a long tail to the left.

**Kurtosis**: This is a measure of the shape of the distribution of the residuals. A normal distribution has a zero measure. A negative value points to a flatter than normal distribution; a positive one has a higher peak than normal distribution.

**Durbin-Watson**: This is a test for the presence of correlation among the residuals. This is especially important for time series modelling

**Jarque-Bera**: This is a combined statistical test of skewness and kurtosis.

**Prob (JB)**: p-value of Jarque-Bera.

**Cond. No**: This is a test for multicollinearity. > 30 indicates unstable results

# statsmodels.formula.api-2

In [ ]:
form_lr = smf.ols(formula = 'y ~ CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT', 
              data=data)
mlr = form_lr.fit()

In [ ]:
mlr.summary()

## Exercise-1

In [ ]:
form_lr = smf.ols(formula = 'y ~ CRIM + ZN + INDUS + CHAS + NOX + RM', 
              data=data)
mlr = form_lr.fit()
mlr.summary()

# Correlation Matrix

Useful diagnostic tool to identify collinearity between predictors



In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
corr_matrix = data.corr()
corr_matrix

In [ ]:
#Display above 60% of the data
corr_matrix[np.abs(corr_matrix) < 0.6] = 0
corr_matrix

In [ ]:
#Visualising above 60% of the data
plt.figure(figsize=(16,10))
sns.heatmap(corr_matrix, annot=True, cmap='YlGnBu')
plt.show()

# Detecting Collinearity with Eigenvectors

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(data.corr())

In [ ]:
pd.Series(eigenvalues).sort_values()

### Note : That index 8, eigenvalue of 0.0635, is near to zero or very small compared to the others. Small value represents presence of collinearity. 

In [ ]:
#arranging Ascending to Decending order
np.abs(pd.Series(eigenvectors[:,8])).sort_values(ascending=False)

### Note : That index 9, 8, 2 have very high loading when compared against the rest

In [ ]:
print(data.columns[2], data.columns[8], data.columns[9])

### These are the factors that are causing multicollinearity problem.

# Revisiting Feature Importance and Extractions

Check:

1. Direction of the coefficient
2. Impact of the variable / factor on the model

## Standardise Variable to Identify Key Feature(s)

In order to perform point 2 properly, one needs to standardise the variable

# LR-Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
model.fit(x,y)

In [ ]:
result = pd.DataFrame(list(zip(model.coef_, data.columns)), columns=['coefficient', 'name']).set_index('name')
np.abs(result).sort_values(by='coefficient', ascending=False)

In [ ]:
from sklearn.preprocessing import StandardScaler  
from sklearn.pipeline import make_pipeline  
scaler = StandardScaler()  
Stand_coef_linear_reg = make_pipeline(scaler, model)

In [ ]:
Stand_coef_linear_reg.fit(x,y)
result = pd.DataFrame(list(zip(Stand_coef_linear_reg.steps[1][1].coef_, data.columns)), 
                      columns=['coefficient', 'name']).set_index('name')
np.abs(result).sort_values(by='coefficient', ascending=False)

# Use $R^2$ to Identify Key Features

* Compare $R^2$ of model against $R^2$ of model without a feature. 

* A significant change in $R^2$ signify the importance of the feature.




In [ ]:
from sklearn.metrics import r2_score

In [ ]:
linear_reg = smf.ols(formula = 'y ~ CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT', 
              data=data)
benchmark = linear_reg.fit()
r2_score(y, benchmark.predict(data))

without LSTAT

In [ ]:
linear_reg = smf.ols(formula = 'y ~ CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B', 
              data=data)
lr_without_LSTAT = linear_reg.fit()
r2_score(y, lr_without_LSTAT.predict(data))

without AGE

In [ ]:
linear_reg = smf.ols(formula = 'y ~ CRIM + ZN + INDUS + CHAS + NOX + RM + DIS + RAD + TAX + PTRATIO + B + LSTAT', 
              data=data)
lr_without_AGE = linear_reg.fit()
r2_score(y, lr_without_AGE.predict(data))

In [ ]:
linear_reg = smf.ols(formula = 'y ~ CRIM + ZN + INDUS + CHAS + NOX + RM + AGE+ RAD + TAX + PTRATIO + B + LSTAT', 
              data=data)
lr_without_DIS = linear_reg.fit()
r2_score(y, lr_without_DIS.predict(data))

Without DIS

In [ ]:
linear_reg = smf.ols(formula = 'y ~ CRIM + ZN + INDUS + CHAS + NOX + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT', 
              data=data)
lr_without_RM = linear_reg.fit()
r2_score(y, lr_without_RM.predict(data))

Without RM

# Gradient Descent

Inspired [Chris McCormick on Gradient Descent Derivation](http://mccormickml.com/2014/03/04/gradient-descent-derivation/)

# Background

$h(x) = \theta_0 + \theta_1X$

Find the values of $\theta_0$ and $\theta_1$ which provide the best fit of our hypothesis to a training set. 

The training set examples are labeled $x$, $y$, 

$x$ is the input value and $y$ is the output. 

The $i$th training example is labeled as $x^{(i)}$, $y^{(i)}$.

## MSE Cost Function

The cost function $J$ for a particular choice of parameters $\theta$ is the mean squared error (MSE):

$$J(\theta)=\frac{1}{m}\sum_{i=1}^m(h_{\theta}(x^{(i)})-y^{(i)})^2$$

$m$ The number of training examples

$x^{(i)}$ The input vector for the $i^{th}$ training example

$y^{(i)}$ The class label for the $i^{th}$ training example

$\theta$ The chosen parameter values of "weights" ($\theta_0, \theta_1, \theta_2$)

$h_{\theta}(x^{(i)})$ The algorithm's prediction for the $i^{th}$ training example using the parameters $\theta$

The MSE measures the mean amount that the model's predictions deviate from the correct values.

It is a measure of the model's performance on the training set. 

The cost is higher when the model is performing poorly on the training set. 

The objective of the learning algorithm is to find the parameters $\theta$ which give the minimum possible cost $J$.

This minimization objective is expressed using the following notation, which simply states that we want to find the $\theta$ which minimizes the cost $J(\theta)$.

$$\min_{\theta}J(\theta)$$

